In [9]:
# Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
import csv
import os

In [7]:
# Loading data
train_path = 'E:/Praca Magisterska/Master-s_Thesis/dane/GTSRB/Train'
test_path = 'E:/Praca Magisterska/Master-s_Thesis/dane/GTSRB/Test'

In [16]:
# function for reading the images
# arguments: path to the traffic sign data, for example './GTSRB/Training'
# returns: list of images, list of corresponding labels 
def readTrafficSigns(rootpath):
    '''Reads traffic sign data for German Traffic Sign Recognition Benchmark.

    Arguments: path to the traffic sign data, for example './GTSRB/Training'
    Returns:   list of images, list of corresponding labels'''
    images = [] # images
    labels = [] # corresponding labels
    # loop over all 42 classes
    for c in range(0,43):
        prefix = rootpath + '/' + format(c, '01d') + '/' # subdirectory for class
        gtFile = open(prefix + format(c, '05d') + '_' + format(c, '05d') + '.csv') # annotations file
        gtReader = csv.reader(gtFile, delimiter=';') # csv parser for annotations file
        gtReader.next() # skip header
        # loop over all images in current annotations file
        for row in gtReader:
            images.append(plt.imread(prefix + row[0])) # the 1th column is the filename
            labels.append(row[7]) # the 8th column is the label
        gtFile.close()
    return images, labels

readTrafficSigns("E:/Praca Magisterska/Master-s_Thesis/dane/GTSRB/Train")

# The German Traffic Sign Recognition Benchmark
#
# sample code for reading the traffic sign images and the
# corresponding labels
#
# example:

# trainImages, trainLabels = readTrafficSigns('GTSRB/Training')
# print len(trainLabels), len(trainImages)
# plt.imshow(trainImages[42])
# plt.show()

In [ ]:
# Loading attached .csv's to get basic info about dataset

#Train.csv contains information regarding each train image
df_train = pd.read_csv("E:/Praca Magisterska/Master-s_Thesis/dane/GTSRB/Train.csv")
labels_train = df_train['ClassId']

#Test.csv contains information regarding each test image
df_test = pd.read_csv('E:/Praca Magisterska/Master-s_Thesis/dane/GTSRB/Test.csv')
labels_test = df_test['ClassId']
df_train.head()